# 1.6 signac-flow HOOMD-blue Example

## About

This notebook contains a minimal example for running a *signac-flow* project from scratch.
The example demonstrates how to compare an ideal gas with a Lennard-Jones (LJ) fluid by calculating a p-V phase diagram.

This examples uses the general-purpose simulation toolkit [HOOMD-blue](http://glotzerlab.engin.umich.edu/hoomd-blue/) for the execution of the molecular-dynamics (MD) simulations.

## Author

Carl Simon Adorf, Bradley Dice

## Before you start

This example requires signac, signac-flow, HOOMD-blue and numpy.
You can install these package for example via conda:
```
conda install -c conda-forge hoomd numpy signac signac-flow
```

In [1]:
import flow
import hoomd
import numpy as np
import signac

# Initialize the HOOMD-blue execution context
hoomd.context.initialize("")

project_root = "projects/tutorial-signac-flow-hoomd"


class MyProject(flow.FlowProject):
    pass

ModuleNotFoundError: No module named 'hoomd'

We want to generate a pressure-volume phase diagram for a Lennard-Jones fluid with molecular dynamics (MD) using the general-purpose simulation toolkit HOOMD-blue (http://glotzerlab.engin.umich.edu/hoomd-blue/).

We start by defining two functions, one for the initialization of our simulation and one for the actual execution.

In [ ]:
from math import ceil


def init(N):
    with hoomd.context.SimulationContext():
        n = ceil(pow(N, 1 / 3))
        assert n**3 == N
        hoomd.init.create_lattice(unitcell=hoomd.lattice.sc(a=1.0), n=n)
        hoomd.dump.gsd("init.gsd", period=None, group=hoomd.group.all())


def sample_lj(N, sigma, seed, kT, tau, p, tauP, steps, r_cut):
    from hoomd import md

    hoomd.init.read_gsd("init.gsd", restart="restart.gsd")
    group = hoomd.group.all()
    hoomd.dump.gsd("restart.gsd", truncate=True, period=100, phase=0, group=group)
    lj = md.pair.lj(r_cut=r_cut, nlist=md.nlist.cell())
    lj.pair_coeff.set("A", "A", epsilon=1.0, sigma=sigma)
    md.integrate.mode_standard(dt=0.005)
    md.integrate.npt(group=group, kT=kT, tau=tau, P=p, tauP=tauP)
    hoomd.analyze.log("dump.log", ["volume"], 100, phase=0)
    hoomd.run_upto(steps)

We want to use **signac** to manage our simulation data and **signac-flow** to define a workflow acting on the data space.

Now that we have defined the core simulation logic above, it's time to embed those functions into a general *workflow*.
For this purpose we add labels and operations with preconditions/postconditions to `MyProject`, a subclass of `flow.FlowProject`.

The `estimate` operation stores an ideal gas estimation of the volume for the given system.
The `sample` operation actually executes the MD simulation as defined in the previous cell.

In [ ]:
@MyProject.label
def estimated(job):
    return "V" in job.document


@MyProject.label
def sampled(job):
    return job.document.get("sample_step", 0) >= 5000


@MyProject.post(estimated)
@MyProject.operation
def estimate(job):
    sp = job.statepoint()
    job.document["V"] = sp["N"] * sp["kT"] / sp["p"]


@MyProject.post(sampled)
@MyProject.operation
def sample(job):
    import hoomd

    with job:
        with hoomd.context.SimulationContext():
            try:
                sample_lj(steps=5000, **job.statepoint())
            finally:
                job.document["sample_step"] = hoomd.get_step()

Now it's time to actually generate some data! Let's initialize the data space!


In [2]:
project = MyProject.init_project(project_root)

# Uncomment the following two lines if you want to start over!
# for job in project:
#     job.remove()

for p in np.linspace(0.5, 5.0, 10):
    sp = dict(N=512, sigma=1.0, seed=42, kT=1.0, p=p, tau=1.0, tauP=1.0, r_cut=2.5)
    job = project.open_job(sp).init()
    with job:
        init(N=sp["N"])

NameError: name 'MyProject' is not defined

The `print_status()` function allows to get a quick overview of our project's *status*:

In [ ]:
project.print_status(detailed=True, parameters=["p"])

The next cell will attempt to execute all eligible operations.

In [ ]:
project.run()

Let's double check the project status.

In [ ]:
project.print_status(detailed=True)

After running all operations we can make a brief examination of the collected data.

In [ ]:
def get_volume(job):
    log = np.genfromtxt(job.fn("dump.log"), names=True)
    N = len(log)
    return log[int(0.5 * N) :]["volume"].mean(axis=0)


for job in project:
    print(job.statepoint["p"], get_volume(job), job.document.get("V"))

For a better presentation of the results we need to aggregate all results and sort them by pressure.

*The following code requires matplotlib.*

In [ ]:
%matplotlib inline
# Display plots within the notebook

from matplotlib import pyplot as plt

V = dict()
V_idg = dict()

for job in project:
    V[job.statepoint()["p"]] = get_volume(job)
    V_idg[job.statepoint()["p"]] = job.document["V"]

p = sorted(V.keys())
V = [V[p_] for p_ in p]
V_idg = [V_idg[p_] for p_ in p]

plt.plot(p, V, label="LJ")
plt.plot(p, V_idg, label="idG")
plt.xlabel(r"pressure [$\epsilon / \sigma^3$]")
plt.ylabel(r"volume [$\sigma^3$]")
plt.legend()
plt.show()

Uncomment and execute the following line to remove all data and start over.

In [ ]:
# %rm -r projects/tutorial-signac-flow-hoomd-blue/workspace